In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
import joblib

publications_df = pd.read_csv('date_propre.csv')
domains_df = pd.read_csv('domaine.csv')

# Crée un dictionnaire pour associer les domaines au panel ERC
domain_to_erc = dict(zip(domains_df['Domaine'], domains_df[ 'Panel ERC']))

# Fonction qui va normaliser les
def clean_and_keep_highest_level_domains(domain_str):
    # Si le domaine n'est pas disponible, on ignore la  o Comment exporter la base de données ? Dans quel format ?

    # On sépare chaque domaine en domaine individuel
    domains = domain_str.split(', ')

    # On extrait le préfixe et le suffixe de chaque domaine
    domain_info = []
    for domain in domains:
        parts = domain.split('.')
        try:
            prefix = int(parts[0])  # le nombre du prefixe
            suffix = '.'.join(parts[1:])  # Le suffixe (e.g., 'spi.nano')
            domain_info.append((prefix, suffix))
        except (ValueError, IndexError):
            continue

    # Si aucun domaine valide n'est trouvé, on retourne une liste vide
    if not domain_info:
        return []

    # On cherche le plus haut préfixe
    max_prefix = max([prefix for prefix, _ in domain_info])

    # On garde seulement les domaines avec les plus hauts préfixes
    highest_level_domains = [suffix for prefix, suffix in domain_info if prefix == max_prefix]

    return highest_level_domains

# On applique la fonction a chaque ligne de la colonne DOMAINE des publications
publications_df['DOMAINE'] = publications_df['DOMAINE'].apply(clean_and_keep_highest_level_domains)

# Fonction qui va associer les domaines aux panels ERC
def map_domains_to_erc(domains):
    erc_panels = set()
    for domain in domains:
        if domain in domain_to_erc:
            erc_panels.update(domain_to_erc[domain].split(', '))
    return list(erc_panels)

# On applique la fonction d'application et on crée une nouvelle colonne 'Panel ERC'
publications_df['Panel ERC'] = publications_df['DOMAINE'].apply(map_domains_to_erc)

publications_df['Panel ERC'] = publications_df['Panel ERC'].apply(lambda x: x if x else ['unknown'])

publications_df['Combined_Features'] = publications_df['DOMAINE'].apply(lambda x: ' '.join(x)) + ' ' + \
                                       publications_df['TITRE'] + ' ' + \
                                       publications_df['RESUME']

publications_df['Combined_Features'] = publications_df['Combined_Features'].fillna('')

tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(publications_df['Combined_Features'])

mlb = MultiLabelBinarizer()
erc_panels_encoded = mlb.fit_transform(publications_df['Panel ERC'])

# Separation des données en données de test et en données d'entrainement
X_train, X_test, y_train, y_test = train_test_split(X, erc_panels_encoded, test_size=0.2, random_state=42)

# Entrainement du modele
model = MultiOutputClassifier(LogisticRegression())
model.fit(X_train, y_train)

# Evaluer le modele
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=mlb.classes_))

# Précision
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

joblib.dump(model, 'erc_panel_predictor.joblib')
joblib.dump(tfidf, 'tfidf.joblib')
joblib.dump(mlb, 'mlb.joblib')

              precision    recall  f1-score   support

           ?       1.00      0.02      0.03        62
         LS1       0.87      0.66      0.75       769
         LS2       0.89      0.61      0.73       700
         LS3       0.90      0.62      0.73       730
         LS4       0.91      0.66      0.77       768
         LS5       0.89      0.59      0.71       720
         LS6       0.90      0.62      0.73       712
         LS7       0.89      0.79      0.83      1031
         LS8       0.91      0.76      0.83      1285
         LS9       0.89      0.67      0.76       845
         PE1       0.99      0.69      0.81       406
        PE10       0.89      0.59      0.71       564
        PE11       0.92      0.65      0.76       394
         PE2       0.88      0.46      0.60       389
         PE3       0.82      0.66      0.74       677
         PE4       0.94      0.68      0.79       317
     PE4,PE5       1.00      0.38      0.55        29
         PE5       0.95    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


['mlb.joblib']

In [10]:
import joblib

model = joblib.load('erc_panel_predictor.joblib')
mlb = joblib.load('mlb.joblib')
tfidf = joblib.load('tfidf.joblib')

new_publication = {
    'DOMAINE': '0.phys, 1.phys.astr, 0.phys, 1.phys.cond',
    #'TITRE': '',,
    #'RESUME': '',
    'RESUME': 'Recherches sur l astrophysique et des matières condensées'
}

new_domains = clean_and_keep_highest_level_domains(new_publication['DOMAINE'])
new_combined_features = ' '.join(new_domains) + ' ' + new_publication['TITRE'] + ' ' + new_publication['RESUME']

new_X = tfidf.transform([new_combined_features])

predicted_panels = model.predict(new_X)

predicted_panel_names = mlb.inverse_transform(predicted_panels)
print(predicted_panel_names)

[('PE3', 'PE9')]


In [ ]:
import pandas as pd
import joblib

# Code pour appliquer le modèle sur un fichier csv de publications.

publications_df = pd.read_csv('publications.csv') # Changer le nom du fichier

model = joblib.load('erc_panel_predictor.joblib')
tfidf = joblib.load('tfidf.joblib')
mlb = joblib.load('mlb.joblib')

def clean_and_keep_highest_level_domains(domain_str):

    if domain_str == 'Domaine non disponible':
        return []

    domains = domain_str.split(', ')

    domain_info = []
    for domain in domains:
        parts = domain.split('.')
        try:
            prefix = int(parts[0])
            suffix = '.'.join(parts[1:])
            domain_info.append((prefix, suffix))
        except (ValueError, IndexError):
            continue

    if not domain_info:
        return []

    max_prefix = max([prefix for prefix, _ in domain_info])

    highest_level_domains = [suffix for prefix, suffix in domain_info if prefix == max_prefix]

    return highest_level_domains

publications_df['DOMAINE'] = publications_df['DOMAINE'].apply(clean_and_keep_highest_level_domains)

publications_df['Combined_Features'] = publications_df['DOMAINE'].apply(lambda x: ' '.join(x)) + ' ' + \
                                       publications_df['TITRE'] + ' ' + \
                                       publications_df['RESUME']

publications_df['Combined_Features'] = publications_df['Combined_Features'].fillna('')

X = tfidf.transform(publications_df['Combined_Features'])

predicted_panels = model.predict(X)

predicted_panel_names = mlb.inverse_transform(predicted_panels)

publications_df['Predicted_Panel_ERC'] = ['; '.join(panels) for panels in predicted_panel_names]

publications_df.to_csv('publications_with_predicted_erc.csv', index=False)